In [25]:
import numpy as np
import pandas as pd
import glob

# Read data
for filename in glob.iglob("sp500_taq/*.dta"):
    # get permnos
    df = pd.read_stata(filename)
    unique_permnos = df.permno.unique()
    with open("permnos.txt", "ab") as f:
        np.savetxt(f, unique_permnos, fmt="%i")
        
# Code has generated permnos.txt file with all the permnos
# sort -u permnos.txt >> output.txt
# This unix command will only save the unique permnos

In [54]:
bdf = pd.read_csv("beta.csv")
bdf['DATE'] = bdf['DATE'].apply(str) # Turn DATE into string

# b_mkt strategy
sdf = pd.DataFrame() # master dataframe

for i in range(1994, 2019):
    if i == 1994:
        for j in range(2,13):
            # code
            year = str(i)
            month = str(j).zfill(2) # Add leading zero if length 1
            date = year + month # ex: 199402
            temp_beta = bdf.loc[bdf['DATE'].str.match(date)].copy()
            temp_beta.loc[:,'quintile'] = pd.qcut(temp_beta['b_mkt'], 5, labels=False, duplicates='drop')
            temp_beta.drop(['n', 'RET', 'alpha', 'b_mkt', 'b_smb', 'b_hml', 'ivol', 'tvol', 'R2', 'exret'], axis=1, inplace=True)
            sdf = sdf.append(temp_beta, ignore_index=True)
    else:
        for j in range(1,13):
            # code
            year = str(i)
            month = str(j).zfill(2) # Add leading zero if length 1
            date = year + month # ex: 199402
            temp_beta = bdf.loc[bdf['DATE'].str.match(date)].copy()
            temp_beta.loc[:,'quintile'] = pd.qcut(temp_beta['b_mkt'], 5, labels=False, duplicates='drop')
            temp_beta.drop(['n', 'RET', 'alpha', 'b_mkt', 'b_smb', 'b_hml', 'ivol', 'tvol', 'R2', 'exret'], axis=1, inplace=True)
            sdf = sdf.append(temp_beta, ignore_index=True)
                                                                                        
sdf['strategy'] = 'b_mkt'
sdf = sdf[['strategy', 'DATE', 'PERMNO', 'quintile']]

# b_smb strategy
tdf = pd.DataFrame() # master dataframe

for i in range(1994, 2019):
    if i == 1994:
        for j in range(2,13):
            # code
            year = str(i)
            month = str(j).zfill(2) # Add leading zero if length 1
            date = year + month # ex: 199402
            temp_beta = bdf.loc[bdf['DATE'].str.match(date)].copy()
            temp_beta.loc[:,'quintile'] = pd.qcut(temp_beta['b_smb'], 5, labels=False, duplicates='drop')
            temp_beta.drop(['n', 'RET', 'alpha', 'b_mkt', 'b_smb', 'b_hml', 'ivol', 'tvol', 'R2', 'exret'], axis=1, inplace=True)
            tdf = tdf.append(temp_beta, ignore_index=True)
    else:
        for j in range(1,13):
            # code
            year = str(i)
            month = str(j).zfill(2) # Add leading zero if length 1
            date = year + month # ex: 199402
            temp_beta = bdf.loc[bdf['DATE'].str.match(date)].copy()
            temp_beta.loc[:,'quintile'] = pd.qcut(temp_beta['b_smb'], 5, labels=False, duplicates='drop')
            temp_beta.drop(['n', 'RET', 'alpha', 'b_mkt', 'b_smb', 'b_hml', 'ivol', 'tvol', 'R2', 'exret'], axis=1, inplace=True)
            tdf = tdf.append(temp_beta, ignore_index=True)

tdf['strategy'] = 'b_smb'
tdf = tdf[['strategy', 'DATE', 'PERMNO', 'quintile']]

# b_hml strategy
rdf = pd.DataFrame() # master dataframe

for i in range(1994, 2019):
    if i == 1994:
        for j in range(2,13):
            # code
            year = str(i)
            month = str(j).zfill(2) # Add leading zero if length 1
            date = year + month # ex: 199402
            temp_beta = bdf.loc[bdf['DATE'].str.match(date)].copy()
            temp_beta.loc[:,'quintile'] = pd.qcut(temp_beta['b_hml'], 5, labels=False, duplicates='drop')
            temp_beta.drop(['n', 'RET', 'alpha', 'b_mkt', 'b_smb', 'b_hml', 'ivol', 'tvol', 'R2', 'exret'], axis=1, inplace=True)
            rdf = rdf.append(temp_beta, ignore_index=True)
    else:
        for j in range(1,13):
            # code
            year = str(i)
            month = str(j).zfill(2) # Add leading zero if length 1
            date = year + month # ex: 199402
            temp_beta = bdf.loc[bdf['DATE'].str.match(date)].copy()
            temp_beta.loc[:,'quintile'] = pd.qcut(temp_beta['b_hml'], 5, labels=False, duplicates='drop')
            temp_beta.drop(['n', 'RET', 'alpha', 'b_mkt', 'b_smb', 'b_hml', 'ivol', 'tvol', 'R2', 'exret'], axis=1, inplace=True)
            rdf = rdf.append(temp_beta, ignore_index=True)

rdf['strategy'] = 'b_hml'
rdf = rdf[['strategy', 'DATE', 'PERMNO', 'quintile']]

# At this point, we have sdf, rdf, tdf
# Merge them together into one master dataframe
mdf = pd.concat([sdf, tdf, rdf], ignore_index=True)
# Rename columns
mdf.rename(columns={'DATE': 'date', 'PERMNO': 'permno', 'quintile': 'portfolio'}, inplace=True)

# Save as csv
mdf.to_csv('mdf.csv', encoding='utf-8', index=False)

,strategy,DATE,PERMNO,quintile
0,b_smb,19940228,10078,2.0
1,b_smb,19940228,10104,3.0
2,b_smb,19940228,10107,2.0
3,b_smb,19940228,10108,4.0
4,b_smb,19940228,10137,3.0
